# Формирования таблицы словаря 'region' для базы данных.

### 1. Подготовка

In [1]:

import time
import json
import datetime

import requests as rq
import pandas as pd
import numpy as np
import tqdm

from sqlalchemy import create_engine
from bs4 import BeautifulSoup as bs


### 2. Формирование переменных для подключения

Справочник было принято взять из википедии

In [ ]:

URL = 'https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%B4%D1%8B_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%\
D1%82%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%\
80%D0%B0%D1%86%D0%B8%D0%B8'

headers = {
    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
    (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36'
}
response_region = rq.get(
    URL,
    headers = headers
)

engine_pet = create_engine('mysql+mysqlconnector://'':''@''/tender_project')


### 3. данные были в теге <'tbody'>

In [3]:

soup_region = (
    bs(response_region.text, 'html.parser')
    .find('tbody')
)


### 4. Были спарсены данные заголовков таблицы, но в дальнейшем было принято решение переименовать данные

In [4]:

l_columns = []

for x in soup_region.find_all('th'):
    l_columns.append(
        ''.join([y.string.strip(' \n') for y in x if y.string is not None]) 
    )
    
l_columns = [
    'Name',
    '_id',
    'OKATO',
    'ISO',
    'GOST'
]


### 5. Тело таблицы 

In [5]:

l_body = []

for x in soup_region.find_all('tr'):
    l_body_x = []
    for y in x.find_all('td'):
        l_body_x.append(
            ''.join(
                [(
                    z
                    .string
                    .strip(' \n')
                    .removeprefix('Оккупационная администрация ')
                ) for z in y if z.string is not None]
            )
        )
    l_body.append(l_body_x)
    
del l_body_x


### 6. Формирование DataFrame region
На один регион может быть несколько значений кода региона, в связи с этим текстовые значения были преобразованы в список и развернуты.
Так как код региона в дальнейшем должне был быть использован как primary key в БД, а в википедии часть кодов регионов могли были дублироваться в разных регионах, в итоге были сгруппированы все значения по коду региона.

In [6]:

df_region = (
    pd
    .DataFrame(
        l_body, 
        columns = l_columns
    )
    .dropna(subset = l_columns)
)

df_region = (
    df_region
    .assign(
        **{
            '_id': df_region['_id'].map(lambda x: x.split(', '))
        }
    )
    .explode('_id')
    .set_index('_id')
)

df_region = df_region.groupby('_id').agg(lambda x: '/'.join(x))


### 7. Загружаем данные во временную таблицу 'tmp_dict_region'

In [7]:

(
    df_region
    .reset_index()
    .to_sql(
        'tmp_dict_region',
        if_exists = 'replace',
        index = False,
        con = engine_pet
    )
)


-1

In [8]:

print(df_region.head())


                           Name OKATO    ISO           GOST
_id                                                        
01   Республика Адыгея (Адыгея)    79  RU-AD  RU-AD, RU-ADY
02      Республика Башкортостан    80  RU-BA  RU-BA, RU-BAS
03           Республика Бурятия    81  RU-BU  RU-BU, RU-BUR
04             Республика Алтай    84  RU-AL  RU-AL, RU-ALI
05          Республика Дагестан    82  RU-DA  RU-DA, RU-DAG
